# Notebook 3 a

In [1]:
import pandas as pd 
from tqdm.auto import tqdm
from pathlib import Path
import pickle
import numpy  as np 
import matplotlib.pyplot as plt
import os

In [2]:
mouse = 'T240'
data_day = '2019_09_27__T240'
date = '2019_09_27'

HowManyBack = 1

Nb2= 'a'
Nb3='a'

data_dir_output = "/Users/gilmandelbaum/Desktop/outPut"

seq_str = "0a1b2a3a4ab5a6abc10a"

In [3]:
# Parameters
data_dir_input = (
    "/Volumes/GilInVivoPaper.large/2.Photometry_in_str/0.data_Sets/1.lateral_medial_6/"
)
data_dir_output = (
    "/Volumes/GilInVivoPaper.large/2.Photometry_in_str/1.lateral_medial_6_analysis/"
)
drop2blocks = "yes"
drop2lastblocks = "yes"
seq_str = "0a1b2b3a4abcd5a6a7a"
HowManyBack = 1
Nb0 = "a"
Nb1 = "b"
Nb2 = "b"
Nb3 = "a"
Nb4 = ["a", "b", "c", "d"]
Nb5 = "a"
Nb6 = "a"
Nb7 = "a"
Tags = [
    "Reward_NoReward_tag",
    "ENL_tag",
    "Cue_to_Sel_tag",
    "Cue_to_Sel_fromPrevTrial_tag",
]
Timing = [600, 200, 15, 15]
Start_or_end = ["start", "end", "start", "start"]
y_axis_list = ["df_f", "z_score", "z_score_sliding_window"]
mouse = "T171"
data_day = "2019_02_20__T171"
date = "2019_02_20"
photo_day = "T171-153511"


In [4]:
root = Path(data_dir_output+"/"+mouse+"/"+data_day+'/'+str(HowManyBack)+"_Back")
d = mouse+"_"+date+"Notebook_2_"+Nb2+'.pickle'
my_path = root / d 
# open a file, where you stored the pickled data
fileToOpen = open(my_path, 'rb')
# dump information to that file
aligned_behav_photo = pickle.load(fileToOpen)

In [5]:
aligned_behav_photo

,nTrial,iBlock,iTrial,iOccurrence,iState_start,iState_end,analog1,analog2,d2 R,d1 R,d2 L,d1 L,Real_nTrials
0,3.0,1.0,3.0,0.0,23.0,29.0,11.0,229.0,298.256378,202.358383,341.851379,156.123901,3
1,3.0,1.0,3.0,1.0,29.0,30.0,9.0,229.0,307.064453,205.430099,347.942444,158.395721,3
2,3.0,1.0,3.0,2.0,30.0,30.0,2740.0,1206.0,304.655029,204.664139,346.281738,157.715408,3
3,3.0,1.0,3.0,3.0,30.0,30.0,2754.0,1207.0,306.073120,205.209686,347.257324,158.047348,3
4,3.0,1.0,3.0,4.0,30.0,30.0,9.0,229.0,305.042480,204.918961,346.540955,157.747101,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
254125,255.0,33.0,22.0,935.0,48.0,48.0,9.0,230.0,NaN,NaN,NaN,NaN,255
254126,255.0,33.0,22.0,936.0,48.0,48.0,11.0,230.0,NaN,NaN,NaN,NaN,255
254127,255.0,33.0,22.0,937.0,48.0,48.0,9.0,230.0,NaN,NaN,NaN,NaN,255
254128,255.0,33.0,22.0,938.0,48.0,48.0,9.0,230.0,NaN,NaN,NaN,NaN,255


# Get df/f for photometry data based on sliding window

In [6]:
photometry_all_channel_afterDownSample = aligned_behav_photo[["d2 R","d1 R","d2 L","d1 L"]]

In [7]:
photometry_all_channel_afterDownSample_c = photometry_all_channel_afterDownSample.copy()
photometry_all_channel_afterDownSample_c.to_dict('list');

In [8]:
# from https://stackoverflow.com/questions/40084931/taking-subarrays-from-numpy-array-with-given-stride-stepsize/40085052#40085052
# dang this is fast!
def strided_app(a, L, S):  # Window len = L, Stride len/stepsize = S
    nrows = ((a.size-L)//S)+1
    n = a.strides[0]
    return np.lib.stride_tricks.as_strided(a, shape=(nrows, L), strides=(S*n, n))

In [9]:
def get_baseline(data, baseline_win, baseline_fun):

    baseline_win_samples = int(baseline_win)
    padded_data = np.pad(data,
                         (baseline_win_samples // 2,
                          baseline_win_samples // 2 - 1),
                         'edge')

    strides = strided_app(padded_data, baseline_win_samples, 1)
    baseline = np.zeros((strides.shape[0], ), dtype='float32')
    baseline[:] = np.nan

    for k, data_slice in tqdm(enumerate(strides), total=strides.shape[0], desc='Computing baseline'):
        use_data = data_slice[np.isfinite(data_slice)]
        if len(use_data) > 0:
            baseline[k] = baseline_fun(use_data)
            
    return baseline

In [10]:
baseline_fun = lambda x: np.percentile(x, 10)

In [ ]:
final_sig = {}

for k, v in photometry_all_channel_afterDownSample_c.items():
    baseline = get_baseline(v, 2000 ,baseline_fun)
    final_sig['{}_f0'.format(k)] = baseline
    final_sig['{}_dff'.format(k)] = (v - baseline) / baseline

In [ ]:
use_keys = [_ for _ in final_sig.keys() if 'dff' in _]

In [ ]:
tstamps = list(range(len(final_sig ["d2 R_dff"])))

# Plot 10 seconds of photometry data to see how it looks. 

In [ ]:
# stash result in a cache directory, for how many back. 
cache_dir_plt = os.path.join(data_dir_output+'/'+mouse+'/'+data_day, str(HowManyBack)+'_Back',"0.plots")
if not os.path.exists(cache_dir_plt):
    os.makedirs(cache_dir_plt)

In [ ]:
path_to_plot = cache_dir_plt+"/"+seq_str[:seq_str.index("4")]+"_plot"

In [ ]:
fig, ax = plt.subplots(4, 1,figsize=(10,10),sharex=True, sharey=False)

for _ax, k in zip(ax.flatten(), use_keys):
    _ax.plot(tstamps, final_sig[k])
    _ax.set_title(k.split('_')[0])
    
_ax.set_xlim(30000,50000)
_ax.set_xlabel('Time(ms)')
_ax.set_ylabel('dF/F0')
fig.savefig(path_to_plot+"_1"+'.pdf')
plt.show()

In [ ]:
fig, ax = plt.subplots(4, 1,figsize=(10,10),sharex=True, sharey=False)

for _ax, k in zip(ax.flatten(), use_keys):
    _ax.plot(tstamps, final_sig[k])
    _ax.set_title(k.split('_')[0])
    
_ax.set_xlim(80000,100000)
_ax.set_xlabel('Time(ms)')
_ax.set_ylabel('dF/F0')
fig.savefig(path_to_plot+"_2"+'.pdf')
plt.show()

In [ ]:
# update photometry signal after df/f 
photometry_all_channel_df0_dff = pd.DataFrame(final_sig)
photometry_all_channel_df0_dff = photometry_all_channel_df0_dff[["d2 R_dff","d1 R_dff","d2 L_dff","d1 L_dff"]]

In [ ]:
actualDffPercent = 100*photometry_all_channel_df0_dff

In [ ]:
aligned_behav_photo_dff = aligned_behav_photo
aligned_behav_photo_dff["d2 R"] = actualDffPercent["d2 R_dff"].values
aligned_behav_photo_dff["d1 R"] = actualDffPercent["d1 R_dff"].values
aligned_behav_photo_dff["d2 L"] = actualDffPercent["d2 L_dff"].values
aligned_behav_photo_dff["d1 L"] = actualDffPercent["d1 L_dff"].values

In [ ]:
processed_data = aligned_behav_photo_dff

In [ ]:
root = Path(data_dir_output+"/"+mouse+"/"+data_day+'/'+str(HowManyBack)+"_Back")
d = mouse+"_"+date+"Notebook_3_a"+'_seq'+seq_str[seq_str.index("2"):seq_str.index("4")]+'.pickle'
my_path = root / d 
my_file = open(my_path, 'wb')
my_file = pickle.dump(processed_data,my_file)